In [15]:
import numpy as np
from scipy.optimize import linprog

def convertToLinearProblem (workerTaskData):
    noOfWorkers, noOfTasks = np.shape(workerTaskData)
    c = np.zeros((noOfWorkers * noOfTasks + 1))
    c[-1] = 1
    # A: matrix for inequality constraints
    A = np.zeros((noOfWorkers, noOfWorkers * noOfTasks + 1))
    for i in range(noOfWorkers):
        # set -T in last column
        A[i, -1] = -1
        for j in range(i * noOfTasks,(i+1)* noOfTasks):
            A[i, j] = 1  
    # b: vector for inequality constraints
    b = np.zeros((noOfWorkers))
    # A_eq: matrix for equality constraints
    A_eq = np.zeros((noOfTasks, noOfWorkers * noOfTasks + 1))
    for j in range(noOfTasks):
        for i in range(noOfWorkers):
            A_eq[j, i * noOfTasks + j] = P[i,j]
    # b_eq: vector for equality constraints
    b_eq = np.ones((noOfTasks))
    # there is no upper bound for x_ij (like the default in scipy)
    # lower bound for x_ij is 0 (like the default in scipy)
    return c, A, b, A_eq, b_eq

def convertXToMatrix(x, noOfWorkers, noOfTasks):
    totalTime = x[-1]
    X = np.zeros((noOfWorkers, noOfTasks))
    for i in range(noOfWorkers):
        for j in range(noOfTasks):
            X[i,j] = x[i* noOfTasks + j]
    return X, totalTime

# load example data (skipping first row and first column)
with open("worker_task_data.csv") as data:
    ncols = len(data.readline().split(','))
workerTaskData = np.loadtxt("worker_task_data.csv", delimiter = ',', skiprows=1, usecols=range(1,ncols))

# convert data into linear problem (variables as defined in SciPy library)
c,A,b,A_eq, b_eq  = convertToLinearProblem(workerTaskData)

# solve the linear problem
result = linprog(c,A,b,A_eq, b_eq)
x = result.x
noOfworkers, noOfTasks = np.shape(workerTaskData)
X, totalTime = convertXToMatrix(x, noOfworkers, noOfTasks)

# print the results
np.set_printoptions(precision=2, suppress=True)
print("Number of hours the workers (rows) spend on the tasks (columns): ")
print(X)
print("Minimal time needed to solve all tasks with crew (in hours): ",totalTime)

Number of hours the workers (rows) spend on the tasks (columns): 
[[0.88 1.25 0.   0.   0.   0.  ]
 [0.   0.   0.   2.13 0.   0.  ]
 [0.   0.   0.96 0.18 0.99 0.  ]
 [0.   0.   2.13 0.   0.   0.  ]
 [0.   0.   0.   1.02 0.   1.11]
 [0.93 0.   0.   0.   1.21 0.  ]]
Minimal time needed to solve all tasks with crew (in hours):  2.1310861423236105
